In [133]:
 cleaning= [[0,  11,  7, 13,  11],
           [5,   0, 13, 15,  15],
           [13, 15,  0, 23,  11],
           [9,  13,  5,  0  , 3],
           [3,  7,   7,  7,   0]
          ]

mixing=[40,35,45,32,50]
matrix=[]
minTime=100000

for a in range(len(cleaning)):    
    for b in range(len(cleaning)):
        if a!=b:  
            for c in range(len(cleaning)):
                if c!=a and c!=b: 
                    for d in range(len(cleaning)):
                        if d!=a and d!=b and d!=c:
                            for e in range(len(cleaning)):
                                if e!=a and e!=b and e!=c and e!=d:
                                    matrixTemp=[a,b,c,d,e]
                                    minTimeTemp=cleaning[b][a]+cleaning[c][b]+cleaning[d][c]+cleaning[e][d]+cleaning[a][e]
                                    #print(matrixTemp)
                                    #print(minTimeTemp)
                                    if minTimeTemp<=minTime:
                                        minTime=minTimeTemp
                                        matrix=matrixTemp
                                        print(matrix)
                                        print(minTimeTemp)
                                        
minimumOverallTime=minTime+sum(mixing)    
print('The minimum time is',minimumOverallTime,'minutes')



                

[0, 1, 2, 3, 4]
43
[0, 1, 2, 4, 3]
43
[0, 1, 3, 4, 2]
43
[0, 1, 4, 2, 3]
41
[1, 4, 2, 3, 0]
41
[2, 3, 0, 1, 4]
41
[3, 0, 1, 4, 2]
41
[4, 2, 3, 0, 1]
41
The minimum time is 243 minutes


In [89]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
from gurobipy import quicksum


try:

    n = 5
    cleaning = np.array([  [0,  11,  7, 13,  11],
                           [5,   0, 13, 15,  15],
                           [13, 15,  0, 23,  11],
                           [9,  13,  5,  0  , 3],
                           [3,  7,   7,  7,   0]
                           ])
    
    mixing=[40,35,45,32,50]
        
    # Create a new model
    model = gp.Model("or1")

    # Create variables
    x = model.addMVar((n,n), vtype=GRB.BINARY, name ="x")
    
    
    # Set objective
    model.setObjective(gp.quicksum(cleaning[i][j]*x[i][j]
        for i in range(n) for j in range(n)), GRB.MINIMIZE)
    
    # Add Constraints
  
    # Each value appears once per row
    model.addConstrs((x[i,:].sum() == 1
                     for i in range(n)), name='R')

    # Each value appears once per column
    model.addConstrs((x[:,j].sum() == 1
                     for j in range(n)), name='C')

    # There must be 5 values
    model.addConstrs((x[:,:].sum() == 5
                     for j in range(n)
                     for i in range(n)))
                     
    #In order for the graph to be connected and make a circle
    for i in range(5):
        for j in range(5):
                model.addConstr(x[j,i]+x[i,j] <= 1)
    
    
    # Optimize model
    model.optimize()
    
    #The array that shows the order that should be followed,so we have the minimum time
    array = x.X
    print(array)
    
    #The minimum time of the optimazation
    minTime=model.ObjVal

    #The overall time adding the mixing time 
    minimumOverallTime=minTime+sum(mixing)    
    print('\nThe overall minimum time is',minimumOverallTime,'minutes')

except GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 60 rows, 25 columns and 720 nonzeros
Model fingerprint: 0x3bb7c52c
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 41.0000000
Presolve removed 40 rows and 5 columns
Presolve time: 0.00s
Presolved: 20 rows, 20 columns, 60 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: objective 3.900000e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0        41.00000   41.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations) in 0.0

In [134]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
from gurobipy import quicksum


try:

    n = 3
    M = 1000  #3M-4M
    m = 7
    paintTime = np.array([  [45,  20,  12],
                           [ 0,  10,  17],
                           [10,  34,  28],
                           ])

        
    # Create a new model
    model = gp.Model("or2")

    # Create variables
    beginningTime   = model.addMVar((n,n), name ="beginningTime")
    y               = model.addMVar(shape=m, vtype=GRB.BINARY, name ="y")
    endTime         = model.addVar(1,name ="endTime")
  
    # Set objective
    objective =  [beginningTime[2,0]+paintTime[2,0],beginningTime[2,1]+paintTime[2,1],beginningTime[1,2]+paintTime[1,2]]
    
    model.setObjective(endTime,GRB.MINIMIZE)
 
    # Add Constraints  
    
    # constraints for machine 1
    model.addConstr(beginningTime[0,0]+paintTime[0,0]<=beginningTime[0,1]+(M*(1-y[0])), "c0")
    model.addConstr(beginningTime[0,1]+paintTime[0,1]<=beginningTime[0,0]+(M*y[0]), "c1")    
    model.addConstr(beginningTime[0,0]+paintTime[0,0]<=beginningTime[0,2]+(M*(1-y[1])), "c2")
    model.addConstr(beginningTime[0,2]+paintTime[0,2]<=beginningTime[0,0]+(M*y[1]), "c3")
    model.addConstr(beginningTime[0,1]+paintTime[0,1]<=beginningTime[0,2]+(M*(1-y[2])), "c4")
    model.addConstr(beginningTime[0,2]+paintTime[0,2]<=beginningTime[0,1]+(M*y[2]), "c5")
    
    # constraints for machine 2
    model.addConstr(beginningTime[1,1]+paintTime[1,1]<=beginningTime[1,2]+(M*(1-y[3])), "c6")
    model.addConstr(beginningTime[1,2]+paintTime[1,2]<=beginningTime[1,1]+(M*y[3]), "c7")
    
    # constraints for machine 3
    model.addConstr(beginningTime[2,0]+paintTime[2,0]<=beginningTime[2,1]+(M*(1-y[4])), "c8")
    model.addConstr(beginningTime[2,1]+paintTime[2,1]<=beginningTime[2,0]+(M*y[4]), "c9")
    model.addConstr(beginningTime[2,0]+paintTime[2,0]<=beginningTime[2,2]+(M*(1-y[5])), "c10")
    model.addConstr(beginningTime[2,2]+paintTime[2,2]<=beginningTime[2,0]+(M*y[5]), "c11")
    model.addConstr(beginningTime[2,1]+paintTime[2,1]<=beginningTime[2,2]+(M*(1-y[6])), "c12")
    model.addConstr(beginningTime[2,2]+paintTime[2,2]<=beginningTime[2,1]+(M*y[6]), "c13")
    
    
    # colors in order wallpaper1
    model.addConstr(beginningTime[0,0]+paintTime[0,0]<=beginningTime[2,0], "c14")
    
    # colors in order wallpaper2
    model.addConstr(beginningTime[1,1]+paintTime[1,1]<=beginningTime[0,1], "c15")
    model.addConstr(beginningTime[0,1]+paintTime[0,1]<=beginningTime[2,1], "c16")
    
    # colors in order wallpaper3
    model.addConstr(beginningTime[2,2]+paintTime[2,2]<=beginningTime[0,2], "c17")
    model.addConstr(beginningTime[0,2]+paintTime[0,2]<=beginningTime[1,2], "c18")
        
    
    # finish constr
    model.addConstr(objective[0]<=endTime, "c19")
    model.addConstr(objective[1]<=endTime, "c20")
    model.addConstr(objective[2]<=endTime, "c21")
            
 
    # Optimize model
    model.optimize()
    
    print('\n\nThe time the process begins = \n',beginningTime.X)
    print('\nThe time the process ends = \n',beginningTime.X+paintTime)
    print('\n\nThe varialbes y =',y.X)
    # The minimum time of the optimazation
    minTime=endTime

    # The overall time adding the mixing time 
    minimumOverallTime=minTime  
    print('\nThe overall minimum time is',minimumOverallTime.X,'minutes')

except GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 17 columns and 58 nonzeros
Model fingerprint: 0x408a8292
Variable types: 10 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 22 rows, 16 columns, 58 nonzeros
Variable types: 9 continuous, 7 integer (7 binary)
Found heuristic solution: objective 142.0000000

Root relaxation: objective 6.400000e+01, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   85.00000    0    4  142.00000   85.00000  40.1%     -    0s
H    0     0                      97.0000000   85.000